# Detección de melanomas malignos utilizando CNN y TPUs

## Librerías a utilizar

In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import seed
seed(1)
tf.random.set_seed(1)

## Configuración de TPU para el entrenamiento </br>


<img src="https://storage.googleapis.com/kaggle-media/tpu/tpu_cores_and_chips.png">

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # busca conectarse a una TPU disponible por grpc
    print('Conexion:', tpu.master()) # muestra el TPU (hostname)
    strategy = tf.distribute.experimental.TPUStrategy(tpu) # instancia una estrategia de distribución
    print('Número de núcleos:', strategy.num_replicas_in_sync) # 4 chips, 2 núcleos c/u
except:
    strategy = tf.distribute.get_strategy() # trabajar con CPU  o GPU
    print('Número de núcleos: ', strategy.num_replicas_in_sync)

### Funcionamiento de un CPU </br>
<img src="https://cloud.google.com/tpu/docs/images/image6.gif"> </br>
### Funcionamiento de un GPU </br>
<img src="https://cloud.google.com/tpu/docs/images/image2.gif"> </br>
### Funcionamiento del TPU </br>
<img src="https://cloud.google.com/tpu/docs/images/image1_2pdcvle.gif"> </br>
Fuente: https://cloud.google.com/tpu/docs/beginners-guide

## Definir ruta en  Google Cloud Storage (necesario para TPU), Batch Size, tamaños de imagen



<img src="https://www.fatalerrors.org/images/blog/d28ad66f32d6730cfcc732d8dcd74c64.jpg">

In [ ]:
!lscpu

In [ ]:
!lsb_release -a

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE # es un comodín que calcula el número óptimo para cierto parámetro en tiempo de ejcución
GCS_PATH = KaggleDatasets().get_gcs_path() # obtener la ruta de la data en GCS
BATCH_SIZE = 128 * strategy.num_replicas_in_sync # batch_size = 128
IMAGE_SIZE = [1024, 1024] # tamaño de las imágenes en tfrec
IMAGE_RESIZE = [256, 256] # tamaño al entrar a la CNN

In [ ]:
GCS_PATH

## Tamaño de la data en .jpeg

In [ ]:
!du -sh ../input/siim-isic-melanoma-classification/jpeg/train

## Tamaño en tfrec

In [ ]:
!du -ach ../input/siim-isic-melanoma-classification/tfrecords/train*

## ¿Por qué TFRecords?

<img src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F4003597%2F9564cf28fc1ec979cbbfa286e115aac2%2FScreen%20Shot%202020-12-01%20at%2017.50.06.png?generation=1606874056869465&alt=media">



## Dividir en data de entrenamiento y validación (a nivel de tfrecs)

In [ ]:
FILENAMES = tf.io.gfile.glob(GCS_PATH + '/tfrecords/train*')
FILENAMES

In [ ]:
len(FILENAMES)

In [ ]:
TRAINING_FILENAMES, VALID_TEST_FILENAMES = train_test_split( # dividir en train y test
    FILENAMES, # busca por patrón
    test_size=0.2, random_state=1 # tamaño del test y un seed
)

In [ ]:
TRAINING_FILENAMES, VALID_TEST_FILENAMES

In [ ]:
VALID_FILENAMES, TEST_FILENAMES = train_test_split( # dividir en train y test
    VALID_TEST_FILENAMES, # busca por patrón
    test_size=0.5, random_state=1 # tamaño del test y un seed
)

In [ ]:
TRAINING_FILENAMES, VALID_FILENAMES, TEST_FILENAMES

In [ ]:
len(TRAINING_FILENAMES), len(VALID_FILENAMES), len(TEST_FILENAMES)

## Contar data para el batching, balance de clases

In [ ]:
train_csv = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')

In [ ]:
!ls /kaggle/input/siim-isic-melanoma-classification/jpeg/train | wc -l

In [ ]:
total = train_csv['target'].size

maligno = np.count_nonzero(train_csv['target'])
benigno = total - maligno

print('Total: {}'.format(total))
print('Maligno: {}, Benigno: {}'.format(maligno, benigno))

## Conteo de número de imágenes de training y validación (necesario para el batching)

In [ ]:
NUM_IMG_TRAIN = 11*2071 + 2061
NUM_IMG_VAL = 2071
NUM_IMG_TEST = 2071

## Ver la estructura de la data en formato tfrec

In [ ]:
primer_tfrec = tf.data.TFRecordDataset(TRAINING_FILENAMES[0]) #primer tfrec

for raw_record in primer_tfrec.take(1): # coge un elemento del tfrec. Take genera un dataset a partir del tfrec
    example = tf.train.Example() #instancia un objeto para leer en formato tfrec (como un JSON)
    example.ParseFromString(raw_record.numpy()) # lee los datos serializados
    print(example) #debemos ver las keys y el tipo de dato

## Crear el pipeline de entrada de datos

<img src="https://storage.googleapis.com/jalammar-ml/tf.data/images/tf.data-pipeline-4.png">

### Convertir imagen .jpeg a tensor y normalizar

In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3) # convierte jpeg en un tensor uint8
    image = tf.cast(image, tf.float32) / 255.0 # normalizar la imagen
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # dar forma (1024, 1024, 3) (aunque ya está así)
    return image

### Leer un ejemplo de entrenamiento en .tfrec

In [ ]:
def read_tfrecord(example):
    tfrec_format = { # obtener del código anterior
        "image": tf.io.FixedLenFeature(shape = (), dtype = tf.string),
        "image_name": tf.io.FixedLenFeature(shape = (), dtype = tf.string), 
        "target": tf.io.FixedLenFeature(shape = (), dtype = tf.int64)
    }
    example = tf.io.parse_single_example(example, tfrec_format) # pasar de serializado a formato
    image = decode_image(example['image']) # decodificar imagen .jpeg a tensor
    label = tf.cast(example['target'], tf.int32)
    return image, label

### Cargar un dataset de .tfrec para luego pasarlo a tensores (con `dataset.map`)

In [ ]:
def load_dataset(filenames):
    opciones = tf.data.Options() # configuración del dataset
    opciones.experimental_deterministic = False # no se tiene un orden específico
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # crea un dataset a paratir de tfrec
    dataset = dataset.with_options(opciones) # usa la data en cualquier orden
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE) # realizar en paralelo
    return dataset

In [ ]:
ds_dummy = load_dataset(TRAINING_FILENAMES)
print(ds_dummy)

In [ ]:
for (image, label) in ds_dummy.take(1):
    print(image, label)

## Obtener el dataset procesado

### Data augmentation (flip)

In [ ]:
def augmentation_pipeline(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.resize(image, IMAGE_RESIZE)
    return image, label

In [ ]:
def valid_test_map(image, label):
    image = tf.image.resize(image, IMAGE_RESIZE)
    return image, label

### Dataset de entrenamiento, con augmentation, repeat, shuffle, batch

<a href="https://ibb.co/DzJSN7C"><img src="https://i.ibb.co/sbfMhFw/repeat-shuffle-batch.png" alt="repeat-shuffle-batch" border="0"></a><br /><a target='_blank' href='https://es.imgbb.com/'>

In [ ]:
def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES)
    dataset = dataset.map(augmentation_pipeline, num_parallel_calls=AUTOTUNE) # se realiza el aumento de datos
    dataset = dataset.shuffle(buffer_size=NUM_IMG_TRAIN, seed=42) # genera un buffer de tamaño 2048 para consumir la data
    dataset = dataset.repeat() # repite los elementos del dataset para que no se acabe la memoria
    dataset = dataset.batch(BATCH_SIZE) # separa el dataset en batches de tamaño BATCH_SIZE, hacerlo después garantiza
    # que los batches sean diferentes en cada epoch
    dataset = dataset.prefetch(AUTOTUNE) # prepara los elementos siguientes mientras se procesan los actuales
    return dataset

### Dataset de validación

In [ ]:
def get_validation_test_dataset(VAL_OR_TEST_FILENAMES):
    dataset = load_dataset(VAL_OR_TEST_FILENAMES)
    dataset = dataset.map(valid_test_map, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()  # almacena data en memoria para reducir algunas operaciones
    dataset = dataset.prefetch(AUTOTUNE) # asegura que haya un batch de data listo antes
    return dataset

In [ ]:
train_dataset = get_training_dataset()
valid_dataset = get_validation_test_dataset(VALID_FILENAMES)
test_dataset = get_validation_test_dataset(TEST_FILENAMES)

## Construcción del modelo

<img src="https://www.researchgate.net/publication/336874848/figure/fig1/AS:819325225144320@1572353764073/Illustrations-of-transfer-learning-a-neural-network-is-pretrained-on-ImageNet-and.png">

<img src="https://1.bp.blogspot.com/-M8UvZJWNW4E/WsKk-tbzp8I/AAAAAAAAChw/OqxBVPbDygMIQWGug4ZnHNDvuyK5FBMcQCLcBGAs/s640/image5.png">

<a href="https://ibb.co/tzdCkyJ"><img src="https://i.ibb.co/Nr4YQwy/depthwise.jpg" alt="depthwise" border="0"></a>

### Definir un bias inicial

$ p_{maligno} = \frac{1}{1+\exp{(-(w\cdot x+b))}} $

Considerando una primera suposición, sin entradas $x$: </br>

$ p_{maligno} = \frac{maligno}{total} = \frac{1}{1+\exp{(-(b))}} $ </br>

Despejando se obtiene:

In [ ]:
initial_bias = np.log([maligno/benigno])
initial_bias

In [ ]:
def make_model(output_bias, metrics = None):    
    output_bias = tf.keras.initializers.Constant(output_bias)
        
    base_model = tf.keras.applications.MobileNetV2(input_shape=(*IMAGE_RESIZE, 3),
                                                include_top=False,
                                                weights='imagenet')
    
    base_model.trainable = False # fija los parámetros de la red
    
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid',
                              bias_initializer=output_bias)
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=metrics)
    
    return model

In [ ]:
STEPS_PER_EPOCH = NUM_IMG_TRAIN // BATCH_SIZE
VALID_STEPS = NUM_IMG_VAL // BATCH_SIZE

### Asignar mayor peso a los ejemplos menos abundantes

In [ ]:
peso_0 = total / benigno # ~1
peso_1 = total / maligno # ~56..

class_weight = {0: peso_0, 1: peso_1}

print('Pesos para clase 0: {:.2f}'.format(peso_0))
print('Pesos para clase 1: {:.2f}'.format(peso_1))

## Matriz de confusión

<img src="https://in2techs.com/wp-content/uploads/2020/09/exampel-2.png"> </br>
Tasa de verdaderos positivos: $tpr = \frac{TP}{TP+FN}$
Tasa de falsos positivos: $fpr = \frac{FP}{FP+TN} $

## ¿Por qué AUC?

<img src="https://glassboxmedicine.files.wordpress.com/2019/02/roc-curve-v2.png"> </br>


In [ ]:
with strategy.scope(): # llamar al tpu
    model = make_model(output_bias = initial_bias, metrics=tf.keras.metrics.AUC(name='auc'))

### Callbacks para guardar el modelo (mejor métrica)

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("melanoma_mobilenetv2.h5",
                                                   monitor = 'val_auc',
                                                   mode = 'max',
                                                    save_best_only=True)

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_dataset, epochs=20,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=valid_dataset,
    validation_steps=VALID_STEPS,
    callbacks=[checkpoint_cb],
    class_weight=class_weight,
)

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.evaluate(valid_dataset)

In [ ]:
model.evaluate(train_dataset, steps = STEPS_PER_EPOCH)

In [ ]:
y = tf.concat([y for x, y in test_dataset], axis=0)
p_class = model.predict(test_dataset)

In [ ]:
y_pred = np.where(p_class > 0.5, 1, 0).reshape(-1)

In [ ]:
y, y_pred

In [ ]:
conf_mat = tf.math.confusion_matrix(y, y_pred, num_classes=2)
conf_mat

In [ ]:
def show_confusion_matrix(conf_mat, labels):
  plt.figure(figsize=(10, 8))
  sns.heatmap(conf_mat, xticklabels=labels, yticklabels=labels, annot =True, fmt= 'd')
  plt.xlabel('Prediccion')
  plt.ylabel('Real')
  plt.show()

In [ ]:
show_confusion_matrix(conf_mat, ('Benigno', 'Maligno'))

In [ ]:
falsos_positivos = 800/(3268+800)
falsos_positivos

In [ ]:
verdaderos_pos = 46/(46+28)
verdaderos_pos

In [ ]:
train_auc = history.history['auc']
val_auc = history.history['val_auc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(train_auc))

#Plotear el auc  de training y validation
plt.figure()
plt.plot(epochs, train_auc)
plt.plot(epochs, val_auc)
plt.title('AUC de entrenamiento y validation')
plt.show()


In [ ]:
fpr, tpr, thresholds = roc_curve(y, p_class)
auc_test = auc(fpr, tpr)

In [ ]:
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='area = {:.3f}'.format(auc_test))
plt.xlabel('Tasa de verdaderos positivos')
plt.ylabel('Tasa de falsos positivos')
plt.title('ROC curve')
plt.legend()
plt.show()

In [ ]:
!ls /kaggle/working